## Assigment 2: Transformers for Translation 🙊


Have you ever wondered how applications like Google Translate or language translation features in social media platforms work? Behind these impressive technologies are sophisticated machine learning models that can understand and translate text between different languages. One of the most powerful and groundbreaking models used for this purpose is the Transformer model.

In this assignment, you will step into the shoes of an AI researcher and engineer to create your own Transformer model for translating text from English to Spanish. This journey will not only enhance your understanding of machine learning and deep learning but also give you hands-on experience with state-of-the-art techniques in natural language processing.

Let's start by downloading important libraries

In [ ]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install bert_score
!pip install rouge_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

For this assignment we are using the Opus Book dataset (read more about it [here](https://huggingface.co/datasets/Helsinki-NLP/opus_books) ). This dataset easily found in huggingface fits perfectly for our machine translation task.

In [ ]:
from datasets import load_dataset

books = load_dataset("opus_books", "en-es")
books = books["train"].train_test_split(test_size=0.2)
train_dataset = books["train"]['translation']
test_size = int(len(books["test"])/2)
val_dataset = books["test"][:test_size]['translation']
test_dataset = books["test"][test_size:]['translation']

Just to have an idea let's have a quick peak on how our dataset looks like

In [ ]:
train_dataset[0]

Since we don't want to take 8 hours training let's trim our dataset a bit (although this might lead to underperformance, feel free to use the complete dataset if you have the computing power).

SUGESTION: start with a small dataset to debug your code and increase it gradually (same applies with number of epochs, batch size, test set size)

In [ ]:
train_dataset = train_dataset[:30000]

### Preprocessing


Preprocessing is an important part of NLP. This allows us to clean and standarize our data.

In [ ]:
import string
import re
def preprocess_data(text):
  """ Method to clean text from noise and standarize text.
  Arguments
  ---------
  text : List of String
      Text to clean
  Returns
  -------
  text : String
      Cleaned and joined text
  """


  text = text.lower()  # make everything lower case
  text = re.sub(r"[^a-zA-ZáéíóúüñÁÉÍÓÚÜÑ\s]", "", text)  # remove any punctuation or special characters without taking off accent letters like ó,í,etc..


  return text

In [ ]:
assert "para ti es una cuestión de tozudez dijo ana de repente al encontrar una palabra que definiera justamente los pensamientos y el sentir de vronsky un calificativo para aquella expresión de su rostro que tanto la irritaba"==preprocess_data("–Para ti es una cuestión de tozudez –dijo Ana de repente, al encontrar una palabra que definiera justamente los pensamientos y el sentir de Vronsky, un calificativo para aquella expresión de su rostro que tanto la irritaba–."), "Check errors in preprocessing"
print("Good Job!")

For an easier training structure, it is useful to format our training and validation sets. The following function should help with this.

In [ ]:
def create_dataset(dataset,source_lang,target_lang):
  """ Method to create a dataset from a list of text.
  Arguments
  ---------
  text : List of String
     Text from dataset
  source_lang : String
     Source language
  target_lang : String
     Target language
  Returns
  -------
  new_dataset : Tuple of String
      Source and target text in format (source, target)
  """
  new_dataset=[]
  #TODO: iterate through dataset extract source and target dataset and preprocess them creating a new clean dataset with the correct format
  for item in dataset:
    source, target = item[source_lang], item[target_lang]
    source = preprocess_data(source)
    target = preprocess_data(target)
    new_dataset.append((source, target))
  return new_dataset

training_set=create_dataset(train_dataset,'en','es')
validation_set=create_dataset(val_dataset,'en','es')
test_set=create_dataset(test_dataset,'en','es')

### Model Creation


Now that our data is ready, we can get started. Let's start by creating our Sequence to Sequence Transformer model.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward,dropout):
        super(TransformerModel, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers,
                                          dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True) # Transformer model with it's attributes (see pytorch documentation)
        self.fc = nn.Linear(d_model, tgt_vocab_size) # Last linear layer

        self.register_buffer("positional_encoding_buffer", self.positional_encoding(d_model))


    def positional_encoding(self, d_model, maxlen = 5000):
        """Method to create a positional encoding buffer.
        Arguments
        ---------
        d_model: int
            Embedding size
        maxlen: int
            Maximum sequence length
        Returns
        -------
        PE: Tensor
            Positional encoding buffer
        """
        pos = torch.arange(0, maxlen).unsqueeze(1)
        denominator = 10000 ** (torch.arange(0, d_model, 2) / d_model)

        #TODO
        PE = torch.zeros((maxlen, d_model))
        PE[:, 0::2] = torch.sin(pos / denominator) # Calculate sin for even positions
        PE[:, 1::2] = torch.cos(pos / denominator) # Calculate cosine odd positions

        PE = PE.unsqueeze(0)  # Add batch dimension

        return PE


    def forward(self, src, tgt, src_mask=None, tgt_mask=None, src_key_padding_mask=None, tgt_key_padding_mask=None):
        """Method to forward a batch of data through the model."""
        src_emb = self.src_embedding(src) + self.positional_encoding_buffer[:, :src.size(1), :]
        tgt_emb = self.tgt_embedding(tgt) + self.positional_encoding_buffer[:, :tgt.size(1), :]


        output = self.transformer(
            src=src_emb,
            tgt=tgt_emb,
            src_mask=src_mask,
            tgt_mask=tgt_mask,
            src_key_padding_mask=src_key_padding_mask,
            tgt_key_padding_mask=tgt_key_padding_mask
        )

        output = self.fc(output)
        return output

    def encode(self, src, src_mask):
        """Method to encode a batch of data through the transformer model."""
        #TODO
        src_emb = self.src_embedding(src) + self.positional_encoding_buffer[:, :src.size(1), :]
        return self.transformer.encoder(src_emb, src_mask)


    def decode(self, tgt, memory,tgt_mask):
        """Method to decode a batch of data through the transformer model."""
        tgt_emb = self.tgt_embedding(tgt) + self.positional_encoding_buffer[:, :tgt.size(1), :]
        return self.transformer.decoder(tgt_emb, memory,tgt_mask)


Now that our model is ready, we still need some methods that will come in handy during training.

In [ ]:
import torch
def create_padding_mask(seq):
  """ Method to create a padding mask based on given sequence.
  Arguments
  ---------
  seq : Tensor
     Sequence to create padding mask for
  Returns
  -------
  mask : Tensor
      Padding mask
  """
  return (seq == 0) #boolean matrix that is True when datapoint is equal to 0

def create_triu_mask(sz):
  """ Method to create a triangular mask based on given sequence. This is used for the tgt mask in the Transformer model to avoid looking ahead.
  Arguments
  ---------
  seq : Tensor
     Sequence to create triangular mask for
  Returns
  -------
  mask : Tensor
      Triangular mask
  """
  # TODO
  #create triangular mask of size sz x sz
  #tranpose mask and cast to float type
  #in pytorch the masked objects expect -inf instead of zero. Replace all 0 for -inf and all 1's for 0's
  mask = torch.triu(torch.ones((sz, sz)), diagonal=1)
  mask = mask.masked_fill(mask == 1, float('-inf')).masked_fill(mask == 0, float(0.0))
  return mask

def tokenize_batch(source, targets,tokenizer):
  """ Method to tokenize a batch of data given a tokenizer.
  Arguments
  ---------
  source : List of String
     Source text
  targets : List of String
     Target text
  tokenizer : Tokenizer
     Tokenizer to use for tokenization
  Returns
  -------
  tokenized_source : Tensor
      Tokenized source text
  """

  tokenized_source = tokenizer(source, padding='max_length', max_length=128, return_tensors='pt', truncation=True)

  tokenized_targets = tokenizer(targets,  padding='max_length', max_length=128, return_tensors='pt',truncation=True)

  return tokenized_source['input_ids'], tokenized_targets['input_ids']


In [ ]:
a=create_triu_mask(5)
b= torch.tensor([[0., float('-inf'), float('-inf'), float('-inf'), float('-inf')],
        [0., 0., float('-inf'), float('-inf'), float('-inf')],
        [0., 0., 0., float('-inf'), float('-inf')],
        [0., 0., 0., 0., float('-inf')],
        [0., 0., 0., 0., 0.]])
assert torch.equal(a,b), "Issues with create_triu_mask"
print("Good Job!")

### Training


In [ ]:
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer=AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')
PAD_IDX = tokenizer.pad_token_id #for padding
BOS_IDX = tokenizer.bos_token_id #for beggining of sentence
EOS_IDX = tokenizer.eos_token_id #for end of sentence

model = TransformerModel(tokenizer.vocab_size, tokenizer.vocab_size,512, 8, 3, 3, 256,0.1).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
loss_function = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

train_loader = torch.utils.data.DataLoader(training_set, batch_size=8, shuffle=True) #change batch size based on your reasources
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=8, shuffle=False) #change batch size based on your reasources

In [ ]:
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer=AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_epoch(model,train_loader,tokenizer):
    model.train()
    losses = 0

    for src, tgt in tqdm(train_loader):
        src, tgt = tokenize_batch(src, tgt, tokenizer)
        src = src.to(device)
        tgt = tgt.to(device)

        tgt_input = tgt[:,:-1]

        src_seq_len = src.shape[1]
        tgt_seq_len = tgt_input.shape[1]

        #TODO
        src_mask = None #creat src_mask this is basically a matrix of 0s of shape Sequence x Sequence (see https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)
        tgt_mask = create_triu_mask(tgt_seq_len).to(device) #create triangular mask for target

        src_padding_mask = create_padding_mask(src).to(device) #create padding mask for src
        tgt_padding_mask = create_padding_mask(tgt_input).to(device) #create padding mask for tgt

        logits = model(src, tgt_input, src_mask=src_mask, tgt_mask=tgt_mask,
                      src_key_padding_mask=src_padding_mask,
                      tgt_key_padding_mask=tgt_padding_mask) #pass it through model

        optimizer.zero_grad()

        tgt_out = tgt[:,1:]
        loss = loss_function(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_loader))


def evaluate(model,val_dataloader ):
    model.eval()
    losses = 0

    # Disable gradient calculation as on evaluate phase
    with torch.no_grad():
      for src, tgt in tqdm(val_dataloader):
          src, tgt = tokenize_batch(src, tgt, tokenizer)
          src = src.to(device)
          tgt = tgt.to(device)

          tgt_input = tgt[:,:-1] ## Remove the last token for the decoder input

          src_seq_len = src.shape[1]
          tgt_seq_len = tgt_input.shape[1]

          src_mask = None  # Source attention mask (None = no masking)
          tgt_mask = create_triu_mask(tgt_seq_len).to(device)  # Create a causal mask to prevent the model from looking at future words


          src_padding_mask = create_padding_mask(src).to(device)  # Padding mask for source (batch, seq)
          tgt_padding_mask = create_padding_mask(tgt_input).to(device)  # Padding mask for target (batch, seq)


          # No transpose needed , forward pass through the model
          logits = model(src, tgt_input, src_mask=src_mask, tgt_mask=tgt_mask,
                          src_key_padding_mask=src_padding_mask,
                          tgt_key_padding_mask=tgt_padding_mask)

          tgt_out = tgt[:,1:]
          loss = loss_function(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
          losses += loss.item()

    return losses / len(list(val_dataloader))

Now we can start training! Keep in mind this code is very demanding computationally, it has been set to 10 epochs (which can take up to 4 hours) but feel free to change this value depending on your resources, in this case the more epochs you can execute the better 😀

In [ ]:
def train(model, epochs, train_loader,validation_loader ):
  for epoch in range(1, epochs+1):
        train_loss = train_epoch(model,train_loader, tokenizer)
        val_loss = evaluate(model,validation_loader)
        print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}"))

train(model, 10, train_loader,validation_loader)

In [ ]:
model_save_path = '/content/drive/MyDrive/Colab Notebooks/NLP - Assignment 2/Model_save/model.pth'

torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

In [ ]:
## load saved model
import torch
from google.colab import drive
from transformers import AutoTokenizer


drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/Colab Notebooks/NLP - Assignment 2/Model_save/model6.pth'

# create tokenizer and model to match training setup
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')

model = TransformerModel(
    tokenizer.vocab_size,
    tokenizer.vocab_size,
    512, 8, 3, 3, 256, 0.1
)

# Load model weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

print(f"Model loaded successfully from: {model_path}")


### Testing


We will use three different evaluation metrics to see our model's test performance: [Bert Score](https://huggingface.co/spaces/evaluate-metric/bertscore), [Meteor](https://huggingface.co/spaces/evaluate-metric/meteor) and [Rouge](https://huggingface.co/spaces/evaluate-metric/rouge). Please access their hugging face documentation to know how to implement them.

In [ ]:
from evaluate import load
bertscore = load("bertscore")
rouge = load('rouge')
meteor = load('meteor')

Implement greedy decode seen in class

In [ ]:
# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask) #pass src through encoder
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = create_triu_mask(ys.size(1)).to(device) #create triangular mask
        out = model.decode(ys, memory, tgt_mask) #pass through decoder

        prob = model.fc(out[:, -1])

        _, next_word = torch.max(prob, dim=1) #get next word based on probabilities (remember to use .item())
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == EOS_IDX:
            break
    return ys

def translate(model: torch.nn.Module, src_sentence: str, tokenizer):
    model.eval()
    src, _ = tokenize_batch(src_sentence, "", tokenizer)
    src = src.to(device)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.float).to(device)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len= int(num_tokens * 1.2 ), start_symbol=BOS_IDX).flatten()
    return tokenizer.decode(tgt_tokens, skip_special_tokens=True)

In [ ]:
print(translate(model, "hello how are you today",tokenizer))

In [ ]:
test_set = test_set[:1000]

In [ ]:
import numpy as np
# you can also trim test_loader
def test(test_loader, model, tokenizer, device, max_length=200):
  """Method to test our model using best score and meteor metric.
  Arguments
  ---------
  test_loader: Dataloader
    Dataloader that holds test set
  model: nn.Module
    trained Machine Translation model
  tokenizer:
  """
  precision = 0
  recall = 0
  f1 = 0
  meteor_metric = 0
  for src, target in test_loader:
    #Use translade method to evaluate our model
    prediction = translate(model, src, tokenizer)
    results_bert = bertscore.compute(predictions=[prediction], references=[target], lang="es") #get results bert
    results_meteor = meteor.compute(predictions=[prediction], references=[target]) #get results meteo
    precision += np.mean(results_bert['precision']) #get precision of results_bert
    recall += np.mean(results_bert['recall']) #get recall of results_bert
    f1 += np.mean(results_bert['f1']) #get f1 of results_bert
    meteor_metric+= results_meteor['meteor'] #get meteor metric of results_meteor
  return precision / len(test_loader), recall / len(test_loader), f1 / len(test_loader), meteor_metric / len(test_loader)

test(test_set, model, tokenizer, device)

## Let's experiment!

Pick 2 experiments out these 3:
1. Play with a hyperparameter of your choice to measure its effect on the translation.

2. Train an inverse translator from spanish to english and compare the performance.

3. Compare the results of your model with the performance of using the T5 pretrained model. This [tutorial](https://huggingface.co/docs/transformers/en/tasks/translation) on using T5 for machine translation might come in handy.





# Experiment 1 - Play with a hyperparameter to measure the effect on the translation

Use different number of epochs (3 and 4)


In [ ]:
import torch
from google.colab import drive
from transformers import AutoTokenizer
from evaluate import load
import numpy as np

drive.mount('/content/drive')

bertscore = load("bertscore")
meteor = load("meteor")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-base')

def load_model(model_path):
    model = TransformerModel(
        tokenizer.vocab_size,
        tokenizer.vocab_size,
        512, 8, 3, 3, 256, 0.1
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print(f" Model loaded successfully from: {model_path}")
    return model


def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask) #pass src through encoder
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = create_triu_mask(ys.size(1)).to(device) #create triangular mask
        out = model.decode(ys, memory, tgt_mask) #pass through decoder

        prob = model.fc(out[:, -1])

        _, next_word = torch.max(prob, dim=1) #get next word based on probabilities (remember to use .item())
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
        if next_word == EOS_IDX:
            break
    return ys


def translate(model: torch.nn.Module, src_sentence: str, tokenizer):
    model.eval()
    src, _ = tokenize_batch(src_sentence, "", tokenizer)
    src = src.to(device)
    num_tokens = src.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.float).to(device)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len= int(num_tokens * 1.2 ), start_symbol=BOS_IDX).flatten()
    return tokenizer.decode(tgt_tokens, skip_special_tokens=True)



def test(test_loader, model, tokenizer, device, max_length=200):
  """Method to test our model using best score and meteor metric.
  Arguments
  ---------
  test_loader: Dataloader
    Dataloader that holds test set
  model: nn.Module
    trained Machine Translation model
  tokenizer:
  """
  precision = 0
  recall = 0
  f1 = 0
  meteor_metric = 0
  for src, target in test_loader:
    #Use translade method to evaluate our model
    prediction = translate(model, src, tokenizer)
    results_bert = bertscore.compute(predictions=[prediction], references=[target], lang="es") #get results bert
    results_meteor = meteor.compute(predictions=[prediction], references=[target]) #get results meteo
    precision += np.mean(results_bert['precision']) #get precision of results_bert
    recall += np.mean(results_bert['recall']) #get recall of results_bert
    f1 += np.mean(results_bert['f1']) #get f1 of results_bert
    meteor_metric+= results_meteor['meteor'] #get meteor metric of results_meteor
  return precision / len(test_loader), recall / len(test_loader), f1 / len(test_loader), meteor_metric / len(test_loader)



test_set = test_set[:1000]


model_paths = {
    "Epoch 3": "/content/drive/MyDrive/Colab Notebooks/NLP - Assignment 2/Model_save/model3.pth",
    "Epoch 4": "/content/drive/MyDrive/Colab Notebooks/NLP - Assignment 2/Model_save/model4.pth"
}

for epoch_name, path in model_paths.items():
    print(f"\n====================== {epoch_name} ======================")
    model = load_model(path)

    # Evaluate metrics
    precision, recall, f1, meteor_score = test(test_set, model, tokenizer, device)
    print(f"  {epoch_name} Results:")
    print(f"  • Precision: {precision:.4f}")
    print(f"  • Recall:    {recall:.4f}")
    print(f"  • F1 Score:  {f1:.4f}")
    print(f"  • METEOR:    {meteor_score:.4f}")

    translation = translate(model, "hello how are you today", tokenizer)
    print(f"hello how are you today --> {translation}")




## Analysis
For this experiment, i trained the model with 3 epochs and 4 epochs to compare it with the base implementation which was using 6 epochs.

At 3 epochs, the model produced the weakest results with a precision of 0.6703, recall of 0.6506, F1 score of 0.6599, and METEOR of 0.1384, generating the translation “cómo te habéis hecho,” which was grammatically valid but semantically incorrect.

At 4 epochs, the scores slightly increased (F1 = 0.6704, METEOR = 0.1529) and the translation “cómo estáis usted” showed partial grammatical understanding, though still inconsistent in formality.

The best performance was achieved is the base implementation with 6 epochs, with precision of 0.7083, recall of 0.6798, F1 score of 0.6934, and METEOR of 0.1722. While the translation “cómo te has dicho” was still semantically inaccurate, it demonstrated better syntactic structure and vocabulary alignment.

Overall, the results indicate that training for more epochs enhances the model’s ability to capture linguistic patterns and produce translations closer to the target language.

Limitation: Due to the lack of GPU resources, I was unable to experiment further with other hyperparameters or train the model beyond 6 epochs, which may have restricted its full performance potential.





# Experiment 2 - Using T5 pretrained model

In [ ]:
!pip install transformers datasets evaluate bert_score -q

import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from evaluate import load
from tqdm import tqdm
import numpy as np

books = load_dataset("opus_books", "en-es")
books = books["train"].train_test_split(test_size=0.2)
test_size = int(len(books["test"]) / 2)
test_dataset = books["test"][test_size:]["translation"]
test_dataset = test_dataset[:1000]

model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate_t5(sentence, model, tokenizer, device, max_length=128):
    if not sentence.strip():
        return ""
    input_text = f"translate English to Spanish: {sentence}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_beams=5,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

semantic_score = load("bertscore")
meteor = load("meteor")

def evaluate_t5(model, tokenizer, dataset, device):
    predictions = []
    references = []

    for item in tqdm(dataset, desc="Evaluating T5"):
        src = item["en"]
        tgt = item["es"]
        if not src.strip():
            continue
        translation = translate_t5(src, model, tokenizer, device)
        if not translation.strip():
            continue
        predictions.append(translation)
        references.append(tgt)

    results_semantic = semantic_score.compute(predictions=predictions, references=references, lang="es")
    results_meteor = meteor.compute(predictions=predictions, references=references)

    print("\n===== T5 Translation Results =====")
    print(f"Semantic Precision: {np.mean(results_semantic['precision']):.4f}")
    print(f"Semantic Recall:    {np.mean(results_semantic['recall']):.4f}")
    print(f"Semantic F1:        {np.mean(results_semantic['f1']):.4f}")
    print(f"METEOR:             {results_meteor['meteor']:.4f}")

    return {
        "Precision": np.mean(results_semantic["precision"]),
        "Recall": np.mean(results_semantic["recall"]),
        "F1": np.mean(results_semantic["f1"]),
        "METEOR": results_meteor["meteor"]
    }

t5_results = evaluate_t5(model, tokenizer, test_dataset, device)

sentence = "hello how are you today"
translation = translate_t5(sentence, model, tokenizer, device)
print(f"EN: {sentence}")
print(f"ES: {translation}")


## Analysis

The base Transformer model got a Precision of 0.7082, Recall of 0.6798, F1 of 0.6934, and METEOR of 0.1722. It translated “hello how are you today” as “cómo te has dicho”, which is partly correct but not natural in Spanish.

The T5 model performed slightly better in semantic Precision (0.7194), Recall (0.7210), and F1 (0.7198) but had a lower METEOR (0.0847). Its translation “Hallo wie sind Sie heute” was wrong since it’s in German instead of Spanish.

Overall, my base model gave a closer Spanish translation, while T5 captured meaning better but failed in language accuracy.

From my understanding, t5-small and t5-base model was trained on translation english to german. This had an impact on the translation result. I could not find an appropriate t5 model for english to spanish.



# Question 1
Draw the structure of the dataset returned by the create_dataset function, providing an example of its elements.

[("Hello", "Hola")]

# Question 2
Write the final shape of the tensor after it passed through the positional_encoding function.

(batch size, seq length, d_model)

# Quesetion 3
What funtion handles the integration of the encoder's outputs into the decoder?

forward()
